<a href="https://colab.research.google.com/github/Ayush-kr-007/Deep-Learning/blob/main/Hyperparameter_Tuning_a_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np

In [35]:
df = pd.read_csv('/content/diabetes.csv')

In [36]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [37]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [38]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [39]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [40]:
X = scaler.fit_transform(X)

In [41]:
X.shape

(768, 8)

In [42]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=1)

In [105]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [44]:
model =  Sequential()
model.add(Dense(32,activation ='relu',input_dim= 8))
model.add(Dense(1,activation ='sigmoid'))
model.compile(optimizer='Adam',loss ='binary_crossentropy',metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.3392 - loss: 0.7717 - val_accuracy: 0.4545 - val_loss: 0.7037
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4631 - loss: 0.6999 - val_accuracy: 0.6558 - val_loss: 0.6498
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6503 - loss: 0.6504 - val_accuracy: 0.7078 - val_loss: 0.6140
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7280 - loss: 0.6117 - val_accuracy: 0.7532 - val_loss: 0.5852
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7258 - loss: 0.5894 - val_accuracy: 0.7792 - val_loss: 0.5652
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7288 - loss: 0.5706 - val_accuracy: 0.7857 - val_loss: 0.5483
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7417 - loss: 0.5399 - val_accuracy: 0.7857 - val_loss: 0.5341
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7592 - loss: 0.5233 - val_accuracy: 0.7922 - val_loss

In [46]:
#  How to select appropriate oprtimizer

In [47]:
!pip install keras-tuner

In [48]:
import keras_tuner as kt

In [49]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

    model.compile(optimizer=optimizer,loss = 'binary_crossentropy',metrics=['accuracy'])

    return model

In [52]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=5)


Reloading Tuner from ./untitled_project/tuner0.json


In [54]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [55]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [57]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [58]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7623 - loss: 0.5152 - val_accuracy: 0.7662 - val_loss: 0.5139
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7748 - loss: 0.4938 - val_accuracy: 0.7857 - val_loss: 0.4980
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7356 - loss: 0.5227 - val_accuracy: 0.7922 - val_loss: 0.4901
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7782 - loss: 0.4885 - val_accuracy: 0.7987 - val_loss: 0.4852
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7939 - loss: 0.4570 - val_accuracy: 0.7922 - val_loss: 0.4812
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7580 - loss: 0.4926 - val_accuracy: 0.7922 - val_loss: 0.4782
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7765 - loss: 0.4656 - val_accuracy: 0.8052 - val_loss: 0.4768
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7966 - loss: 0.4453 - val_accuracy: 0.79

In [75]:
def built_model(hp):

  model = Sequential()

  units = hp.Int('units',min_value= 8,max_value=128)

  model.add(Dense(units =units,activation = 'relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss = 'binary_crossentropy',metrics=['accuracy'])

  return model

In [76]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'krk')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [77]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.6753246784210205

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 12s


In [78]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [79]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [80]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7334 - loss: 0.5231   
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7478 - loss: 0.4878 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7375 - loss: 0.5017 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7722 - loss: 0.4704 
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7840 - loss: 0.4623 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7464 - loss: 0.4830 
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7628 - loss: 0.4819 
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7536 - loss: 0.4854 
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7759 - loss: 0.4659 
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7871 - loss: 0.4465 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7676 - loss: 0.4638 
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3

<!-- how to select number of layer -->

 how to select number of layers  -->



In [89]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layer',min_value=1,max_value=100)):
    model.add(Dense(72,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [90]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials =3,
                        directory='mydir',
                        project_name='num_layers')

Reloading Tuner from mydir/num_layers/tuner0.json


In [91]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 24s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 02m 02s


In [92]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 88}

In [94]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 182 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [95]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 98ms/step - accuracy: 0.6356 - loss: 0.6627 - val_accuracy: 0.6429 - val_loss: 0.6595
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6829 - loss: 0.6260 - val_accuracy: 0.6429 - val_loss: 0.6532
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6132 - loss: 0.6670 - val_accuracy: 0.6429 - val_loss: 0.6532
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6632 - loss: 0.6401 - val_accuracy: 0.6429 - val_loss: 0.6555
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6643 - loss: 0.6408 - val_accuracy: 0.6429 - val_loss: 0.6538
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6327 - loss: 0.6588 - val_accuracy: 0.6429 - val_loss: 0.6525
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6779 - loss: 0.6303 - val_accuracy: 0.6429 - val_loss: 0.6519
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.6265 - loss: 0.6610 - val_accura

# Final code

In [111]:
def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layer',min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int('units' + str(i),min_value=8,max_value=128,step=8),activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])))
      model.add(Dropout(hp.Choice('dropout' + str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(Dense(hp.Int('units' + str(i),min_value=8,max_value=128,step=8),activation = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])))
      model.add(Dropout(hp.Choice('dropout' + str(i),values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))


    counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                loss = 'binary_crossentropy',
                metrics=['accuracy'])

  return model

In [112]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='my_dir',
                        project_name='diabetes_tuning1')

In [113]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.4675324559211731

Best val_accuracy So Far: 0.6948052048683167
Total elapsed time: 00h 00m 25s


In [114]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 4,
 'units0': 16,
 'activation': 'relu',
 'dropout0': 0.1,
 'optimizer': 'sgd',
 'units1': 8,
 'dropout1': 0.1,
 'units2': 8,
 'dropout2': 0.1,
 'units3': 8,
 'dropout3': 0.1}

In [115]:
model =tuner.get_best_models(num_models=1)[0]

In [116]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.6535 - loss: 0.6740 - val_accuracy: 0.6753 - val_loss: 0.6620
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6665 - loss: 0.6710 - val_accuracy: 0.6753 - val_loss: 0.6581
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6665 - loss: 0.6626 - val_accuracy: 0.6558 - val_loss: 0.6548
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6833 - loss: 0.6570 - val_accuracy: 0.6494 - val_loss: 0.6517
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6991 - loss: 0.6475 - val_accuracy: 0.6623 - val_loss: 0.6489
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7038 - loss: 0.6457 - val_accuracy: 0.6429 - val_loss: 0.6467
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6729 - loss: 0.6513 - val_accuracy: 0.6429 - val_loss: 0.6447
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6798 - loss: 0.6487 - val_accuracy: